In [18]:
# Package imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
np.random.seed(1) # set a seed so that the results are consistent

In [2]:
def sigmoid(z):

    s = 1/(1 + np.exp(-z))
    return s

In [3]:
# Function for setting the structure of Network
def layer_sizes(X, Y):
    
    n_x = X.shape[0]  #Input layer nodes  
    n_h = 4 #total hidden layer
    n_y = 6 #output layer nodes
    
    return (n_x, n_h, n_y)

In [4]:
# Initializing the parameters
def initialize_parameters(n_x, n_h, n_y):
        
    np.random.seed(2) # we set up a seed so that your output matches ours although the initialization is random.
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    
    assert (W1.shape == (n_h, n_x)) ##
    assert (b1.shape == (n_h, 1))  ## To confirm the shapes of Parameters
    assert (W2.shape == (n_y, n_h)) ## As these are the main reason of most errors
    assert (b2.shape == (n_y, 1))  ##
    
    parameters = {"W1": W1, ##
                  "b1": b1, ## Putting all parameters together so that
                  "W2": W2, ## we do not end up with confiusing due to many parameters
                  "b2": b2} ##
    
    return parameters

In [186]:
# forward propagation
def forward_propagation(X, parameters):
    
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters['W1']
    W2 = parameters['W2']
    b1 = parameters['b1']
    b2 = parameters['b2']
   
    # Implement Forward Propagation to calculate A2 (probabilities)
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    #assert(A2.shape == (6, X.shape[1]))
    
    cache = {"Z1": Z1,  
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [187]:
def compute_cost(A2, Y, parameters):
    
    m = Y.shape[1] # number of example

    # Compute the cross-entropy cost
    cost = -np.sum((Y * np.log(A2) + (1 - Y) * np.log(1 - A2)))/m
    
    cost = np.squeeze(cost)     # makes sure cost is the dimension we expect. 
                                # E.g., turns [[17]] into 17 
    assert(isinstance(cost, float))
    
    return cost

In [188]:
#backward Propagation
def backward_propagation(parameters, cache, X, Y):
    
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    W1 = parameters['W1']
    W2 = parameters['W2']
        
    # Retrieve also A1 and A2 from dictionary "cache".
    A1 = cache['A1']
    A2 = cache['A2']
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    dz2 = A2 - Y
    dW2 = np.dot(dz2, A1.T)/m
    db2 = np.sum(dz2, axis = 1, keepdims = True)/m
    
    dz1 = np.dot(W2.T, dz2) * (1 - np.power(A1, 2))
    dW1 = np.dot(dz1, X.T)/m
    db1 = np.sum(dz1, axis = 1, keepdims = True)/m
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [189]:
def update_parameters(parameters, grads, learning_rate = 1.2):
    
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters['W1']
    W2 = parameters['W2']
    b1 = parameters['b1']
    b2 = parameters['b2']
    
    # Retrieve each gradient from the dictionary "grads"
    dW1 = grads['dW1']
    dW2 = grads['dW2']
    db1 = grads['db1']
    db2 = grads['db2']    
    
    # Update rule for each parameter
    W1 = W1 - (learning_rate * dW1)
    W2 = W2 - (learning_rate * dW2)
    b1 = b1 - (learning_rate * db1)
    b2 = b2 - (learning_rate * db2)
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [190]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=True):
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Initialize parameters, then retrieve W1, b1, W2, b2. Inputs: "n_x, n_h, n_y". Outputs = "W1, b1, W2, b2, parameters".
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X, parameters)
        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A2, Y, parameters)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads)
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [191]:
def predict(parameters, X):
    
    # Computes probabilities using forward propagation, and classifies.
    train_y, cache = forward_propagation(X, parameters)
    predicted_y = np.zeros_like(train_y)
    max_term = train_y.argmax(0)

    for i in range(X.shape[1]):
        predicted_y[max_term[i], i] = 1
    
    ratings = np.array(['Not rated', 'Poor', 'Average', 'Good', 'Very Good', 'Excellent'])
    predicted_ratings = np.zeros((1, X.shape[1]))
    predicted_ratings = pd.Series(ratings[predicted_y.argmax(0)])
    
    return predicted_y, predicted_ratings
    

## Above were for trainig the data

## Below cells are for Preprocessing the data 

In [192]:
def preprocessing(data):
    
    
    y = data['Rating text']
    X = data.copy()

    text_col = ['Restaurant Name', 'City', 'Address', 'Cuisines', 'Rating color', 'Rating text', 'Currency',
                'Locality Verbose', 'Locality']
    for i in text_col:
        del X[i]

    ratings = ['Not rated', 'Poor', 'Average', 'Good', 'Very Good', 'Excellent']

    print("Type of ratings : ", ratings)

    
    for i in range(len(ratings)):
        y = y.replace(to_replace = ratings[i], value= i)

    sm_column = ['Has Table booking', 'Has Online delivery', 'Switch to order menu', 'Is delivering now']
    
    
    for i in sm_column:
        X[i] = X[i].replace(to_replace = 'Yes', value = 1)
        X[i] = X[i].replace(to_replace = 'No', value = 0)

    
    #Feature scaling of data using
    scaler = StandardScaler()
    x_scaler = scaler.fit_transform(X)
    
    
    new_data = pd.DataFrame(x_scaler, columns= X.columns)
    new_data['Rating'] = y
    
    return new_data
    

In [193]:
def dataset(data):

    new_data = preprocessing(data)
    
# Splitting The training data and test data
    X = np.array(new_data.iloc[:, :new_data.shape[1] - 1]).T
    n, m = X.shape
    
    X_train = X[:, :6999]
    X_test = X[:, 7000: m-1]
    
    # encoding the data for multiclassification
    
    Y = np.zeros((6,6))
    diag = np.eye(6)
    y = np.array(new_data.iloc[:, new_data.shape[1] - 1], dtype = int)
    Y = diag[y].T
    
    Y_train = Y[:, :6999]
    Y_test = Y[:, 7000: m-1]
    
# Text data to match at the end and getting Accuracy
    y_text_train = data['Rating text'][:6999]
    y_text_train = y_text_train.reshape(y_text_train.shape[0], 1)

    y_text_test = data['Rating text'][7000: 9550]
    y_text_test = y_text_test.reshape(y_text_test.shape[0], 1)
    
    print(m, n)

    return X_train, Y_train, X_test, Y_test, y_text_train, y_text_test

In [194]:
data = pd.read_csv('zomato.csv')
X_train, Y_train, X_test, Y_test, y_text_train, y_text_test = dataset(data)
print("X_train shape : ", X_train.shape, type(X_train))
print("X_test shape : ", X_test.shape, type(X_test))
print("Y_train shape : ", Y_train.shape, type(X_train))
print("Y_test shape : ", Y_test.shape, type(X_test))
print("y_text_train shape : ", y_text_train.shape, type(X_train))
print("y_text_test shape : ", y_text_test.shape, type(X_test))

('Type of ratings : ', ['Not rated', 'Poor', 'Average', 'Good', 'Very Good', 'Excellent'])
(9551, 12)
('X_train shape : ', (12, 6999), <type 'numpy.ndarray'>)
('X_test shape : ', (12, 2550), <type 'numpy.ndarray'>)
('Y_train shape : ', (6, 6999), <type 'numpy.ndarray'>)
('Y_test shape : ', (6, 2550), <type 'numpy.ndarray'>)
('y_text_train shape : ', (6999, 1), <type 'numpy.ndarray'>)
('y_text_test shape : ', (2550, 1), <type 'numpy.ndarray'>)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


## Let's start training the parameters and predict the data, training of the parameters will take 3 to 4 minutes 

In [152]:
# Train the parameters
parameters = nn_model(X_train, Y_train, n_h = 4, num_iterations = 10000, print_cost=True)


Cost after iteration 0: 4.158918
Cost after iteration 1000: 0.230243
Cost after iteration 2000: 0.184716
Cost after iteration 3000: 0.167386
Cost after iteration 4000: 0.162962
Cost after iteration 5000: 0.160279
Cost after iteration 6000: 0.158443
Cost after iteration 7000: 0.157080
Cost after iteration 8000: 0.156012
Cost after iteration 9000: 0.155146


In [195]:
predicted_y_train, predicted_ratings_train = predict(parameters, X_train)
predicted_ratings_train = predicted_ratings_train.values.reshape(predicted_ratings_train.shape[0], 1)

predicted_y_test, predicted_ratings_test = predict(parameters, X_test)
predicted_ratings_test = predicted_ratings_test.values.reshape(predicted_ratings_test.shape[0], 1)

In [196]:
print('Accuracy on training set : ', np.mean(predicted_ratings_train == y_text_train) * 100)
print('Accuracy on training set : ', np.mean(predicted_ratings_test == y_text_test) * 100)

('Accuracy on training set : ', 96.82811830261467)
('Accuracy on training set : ', 96.50980392156863)



### A file named 'new.csv' in folder has the preprocessed data of 'zomato.csv'. For Giving you own data to predict. Use data from 'new.csv'

In [224]:
new_data = pd.read_csv('new.csv', header = None)
X_new_data = np.array(new_data.iloc[:, :new_data.shape[1] - 1].T)
#real_y = np.array(new_data.iloc[:, new_data.shape[1] -1], dtype = str)

In [227]:
real_y = data['Rating text']
# Example of precdition
i = 7000
a, prediction = predict(parameters, X_new_data[:, i].reshape(12,1))
prediction = np.array(prediction)
print('prediction : ' + prediction)
print('real value : ' + real_y[i])


['prediction : Good']
real value : Good


### Now try yourself by changing the value of i